# Init

In [1]:
from decouple import Config, RepositoryEnv
from ollama_interaction import embeding_ollama_request,generate_ollama_request

import pandas as pd
import json

In [2]:
config = Config(RepositoryEnv('.env'))

ollama_api_url = config('OLLAMA_API_URL')
ollama_bearer_token = config('OLLAMA_BEARER_TOKEN')

In [32]:
# context option are based on https://github.com/NVIDIA/RULER
# If model is not on doc we take the nearest one

# best context is based on  Effective length
# max context is base on claimed length

with open('model-data.json','r') as file:
    model_data = json.load(file)

for item in list(model_data.keys()):
    print('model :',item)

model : mistral-nemo:latest
model : mistral-small:latest
model : qwen2.5:14b
model : qwen2.5:32b
model : llama3.2:1b
model : llama3.1:latest


# Scoring generation

In [33]:
data = pd.read_csv("hard-database/data_at_select_ai.csv",index_col='id')

data = data[(data['token_numb_description']<5000)&(data['token_numb_description']>500)]

In [70]:
def gen_prompt_sub_scoring(sub_description,project_description,max_score=5,min_score=-5):
    prompt_system = """Tu es une IA qui aide l'utilisateur à déterminer la compatibilité d'une aide ou subvention par rapport à la description de son projet.
    
    Tu ne dois pas être bavard et te conformer totalement et strictement aux demandes de l'utilisateur.
    
    Ne fournis aucune explication, aucun mot supplémentaire, ni aucun caractère additionnel dans ta réponse.
    """
    
    prompt_user = f"""**Instructions :**

    Attribue une note sous forme d'entier entre {min_score} et {max_score}, selon les critères suivants :

    {max_score} : L'aide ou la subvention correspond parfaitement aux objectifs et aux besoins du projet.
    Les objectifs de l'aide/subvention sont totalement alignés avec les besoins du projet.
    Les conditions et exigences de l'aide/subvention sont complètement satisfaites par le projet.

    {int(max_score*0.2)} : L'aide ou la subvention correspond très bien aux objectifs et aux besoins du projet.
    Les objectifs de l'aide/subvention sont majoritairement alignés avec les besoins du projet.
    La plupart des conditions et exigences de l'aide/subvention sont satisfaites par le projet.

    {0} : L'aide ou la subvention correspond partiellement aux objectifs et aux besoins du projet.
    Les objectifs de l'aide/subvention sont partiellement alignés avec les besoins du projet.
    Certaines conditions et exigences de l'aide/subvention sont satisfaites par le projet.

    {int(min_score*0.2)} : L'aide ou la subvention a une correspondance minimale avec les objectifs et les besoins du projet.
    Les objectifs de l'aide/subvention sont rarement alignés avec les besoins du projet.
    Peu de conditions et exigences de l'aide/subvention sont satisfaites par le projet.

    {min_score} : La description du projet est insuffisamment détaillée ou trop vague, ou l'aide ou la subvention n'a aucune correspondance avec les objectifs et les besoins du projet.
    Les objectifs de l'aide/subvention ne sont pas du tout alignés avec les besoins du projet.
    Aucune des conditions et exigences de l'aide/subvention n'est satisfaite par le projet.

    Analyse les informations suivantes pour déterminer la compatibilité entre l'aide ou la subvention et le projet de l'utilisateur.
    
    - **Aide ou subvention à analyser :**
    {sub_description}
    
    - **Projet de l'utilisateur :**
    {project_description}
    
    **Instructions :**

    - Utilise ces informations pour attribuer une note selon les critères précédemment définis. **Ne me renvoie que la note sans aucunes explications.**"""

    return prompt_system, prompt_user

In [37]:
with open("project-description-sample.json",'r') as file:
    project_descrpition_list = json.load(file)

In [97]:
seed_number = 5
max_retry = 10
row_list = []

model = "llama3.1:latest"
model_options = model_data[model]

score_sub_request_options = {
    "num_ctx": 16384,
    "num_predict": 2
}

error_list = []

request_options = score_sub_request_options

for project_descrpition_key in project_descrpition_list:
    project_description = project_descrpition_list[project_descrpition_key]
    print('-------------------------')
    print(project_description)
    for i, row in data.iterrows():
        print('---------')
        sub_description = row['description']
        sub_eligibility = row['eligibility']
        prompt_system,prompt_user = gen_prompt_sub_scoring(sub_description,project_description,max_score=5,min_score=-5)

        score_sub = 0
        seed = 0
        scoring_made = 0
        retry = 0
        while scoring_made < seed_number and retry < max_retry:
            seed += 1
            response = generate_ollama_request(
                prompt_system=prompt_system,
                response_format=None,
                prompt_user=prompt_user,
                ollama_api_url=ollama_api_url,
                bearer_token=ollama_bearer_token,
                model_options = model_options,  # Default to None
                request_options= request_options,  # Default to None
                seed=seed,
                )
            if response:
                try :
                    response_filtred = response['response'].replace(' ','').replace('\n','')
                    score_seed = int(response_filtred)
                    if score_seed > 5:
                        score_seed =5
                    if score_seed < -5:
                        score_seed = -5
                    score_sub+=score_seed
                    retry = 0
                    scoring_made+=1
                    print(score_sub)
                except Exception as error:
                    retry += 1
                    error_list.append(response_filtred)
                    print(f"error : {response['response']}")
        row['project'] = project_description
        row['project_score'] = score_sub
        row['scoring_made'] = scoring_made
        row['scoring_error'] = seed - scoring_made
        row_list.append(row)

-------------------------
Revitalisation d'une zone humide
---------
1
0
-1
-1
-2
---------
error : 0 (-
-1
-6
-5
-10
-10
---------
0
-1
-2
2
1
---------
1
0
-1
4
3
---------
0
-1
-2
-2
-3
---------
0
-1
-6
-5
-10
---------
1
0
-5
-1
-6
---------
1
0
-1
4
-1
---------
4
3
2
3
2
---------
1
-4
-9
-4
-9
---------
error : 0 (-
-1
-2
-1
-2
-1
---------
0
-5
-10
-10
-15
---------
0
-1
-2
2
1
---------
0
-1
-2
2
1
---------
1
0
-5
0
-5
---------
error : 0 (-
-1
-2
3
2
2
---------
1
0
-1
4
3
---------
4
3
-2
3
-2
---------
0
-1
-6
-5
-10
---------
1
0
-5
-1
-6
---------
1
0
-1
4
-1
---------
1
0
-5
0
-5
---------
4
3
-2
3
-2
---------
0
-1
-6
-1
-6
---------
3
2
-3
1
-4
---------
0
-1
-2
-1
-2
---------
1
0
-1
4
3
---------
0
-1
-2
3
2
---------
1
0
-1
0
-1
---------
4
3
2
7
2
---------
1
-4
-9
-4
-9
---------
5
4
3
7
6
---------
error : 0 (-
-1
-2
-1
-2
-1
---------
4
-1
-6
-1
-6
---------
1
-4
-9
-9
-14
---------
1
0
-5
-10
-15
---------
error : 0 (-
-1
-2
-1
-2
-2
---------
error : 0 (-
-5

In [98]:
data_project_score= pd.DataFrame(row_list)

In [103]:
data_project_score.to_csv("hard-database/data_project_scoring.csv")